In [1]:
# import 套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 匯入資料相對路徑
import os
path=os.path.abspath('.') +'\train_712.csv'
path2=os.path.abspath('.') +'\test_179.csv'
train=pd.read_csv(open(path))
test=pd.read_csv(open(path2))

In [3]:
train=pd.read_csv(r'C:\Users\6506\Desktop\簡報製作\集成法\train_712.csv')
test=pd.read_csv(r'C:\Users\6506\Desktop\簡報製作\集成法\test_179.csv')

In [4]:
# 進行資料清理程序
def dataclean(df):
    # 找出姓名頭銜並分類
    df['Name']=df['Name'].str.split(' ').str[1]
    for i in range(len(df)):
        if df.loc[i,'Name']!='Mr.' and df.loc[i,'Name']!='Matser.' and df.loc[i,'Sex']=='male':
            df.loc[i,'Name']='Other M'
        if df.loc[i,'Name']!='Mrs.' and df.loc[i,'Name']!='Miss.' and df.loc[i,'Sex']=='female':
            df.loc[i,'Name']='Other F'
    # 分類年齡是否為NULL，若為NULL則以中位數代替
    df['Age_na']=df['Age'].isnull()
    df['Age_na'][df['Age_na']]=1
    df['Age'][df['Age'].isnull()]=df['Age'].median()
    # 去除不需要之欄位
    df=df.drop(['Cabin','Ticket','PassengerId'],axis=1)
    # 將類別變數轉換為onehot encoding
    d=df.dtypes==object
    for i in ('Pclass',d[d==True].index[0:]):
        dummies=pd.get_dummies( df[i], prefix=i)
        df = pd.concat( [df, dummies], axis = 1 )
        df=df.drop(i,axis=1)
    return df

In [5]:
# 分別對train、test執行程序
train=dataclean(train)
test=dataclean(test)

C:\Users\6506\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\6506\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\6506\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\6506\Ap

In [6]:
# random code
RS=42
#decision tree
from sklearn.tree import DecisionTreeClassifier
#naive bayes
from sklearn.naive_bayes import GaussianNB
#KNN
from sklearn.neighbors import KNeighborsClassifier
#AdaBoost,RandomForest
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
classifiers = [
        ['Cart',DecisionTreeClassifier(max_depth=10, random_state=RS)],        
        ['NBayes',GaussianNB()],
        ['KNN', KNeighborsClassifier(5)],
        ['RandomForest',RandomForestClassifier(n_estimators=100,random_state=RS)],
        ['RandomForest_2',RandomForestClassifier(n_estimators=100,random_state=RS,max_features=2)],
        ['AdaBoost',AdaBoostClassifier(n_estimators=100,random_state=RS)],
]
# 使用前三種分類器Cart、NBayes、KNN實作投票法
for i in range(3):
    clf=classifiers[i][1].fit(train.iloc[:,1:], train.iloc[:,0])
    test['pred_'+classifiers[i][0]]=clf.predict(test.iloc[:,1:19])
test['vote']=round(test.iloc[:,-3:].mean(axis=1)).astype(int)

In [7]:
# 實作兩種Bagging與Boosting
for i in range(3,len(classifiers)):
    clf=classifiers[i][1].fit(train.iloc[:,1:], train.iloc[:,0])
    test['pred_'+classifiers[i][0]]=clf.predict(test.iloc[:,1:19])

In [35]:
# 使用交叉驗證方法，將資料分為k-fold
from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator, TransformerMixin,ClassifierMixin, clone
# BaseEstimator:Base class for all estimators in scikit-learn
# TransformerMixin:Mixin class for all transformers in scikit-learn.
# ClassifierMixin:Mixin class for all classifiers in scikit-learn.
# Clone:a deep copy of the model in an estimator without actually copying attached data. 
#   It yields a new estimator with the same parameters that has not been fit on any data.

# 建立stack類別
class StackingAveragedModels(BaseEstimator, ClassifierMixin, TransformerMixin):
    # 代入Level 0、Level 1模型，fold預設為3
    def  __init__ (self, base_models, meta_model, n_folds=3 ):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds

    # 將原來的模型clone，進行fit功能
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=RS)

        # 對於每個模型，使用交叉驗證方法來訓練初級學習器，並且得到次級訓練集 
        # 建立空物件以輸入次級訓練集
        out_of_fold_predictions = np.zeros((len(X), len(self.base_models)))
        # 使用給定 Level 0 模型分別對 3-fold 訓練集預測
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                # 分別clone為給定模型，進行fit與predict
                instance = clone(model)                                            
                instance.fit(X.iloc[train_index,:], y.iloc[train_index])
                # 存入模型   
                self.base_models_[i].append(instance) 
                y_pred = instance.predict(X.iloc[holdout_index,:])
                # 將預測結果輸入次級訓練集
                out_of_fold_predictions[holdout_index, i] = y_pred

        # 使用次級訓練集來訓練次級學習器
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    # 在上面的fit方法當中，已經將訓練出來的初級學習器和次級學習器保存下來
    # predict的時候只需要用這些學習器再構造次級預測數據集並且進行預測
    def predict(self, X):
        meta_test = np.column_stack([
            np.round(np.column_stack([model.predict(X) for model in base_models]).mean(axis=1 ))
             for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_test)

In [36]:
# 實作Stacking
clf=StackingAveragedModels(base_models=[classifiers[i][1] for i in range(3)],meta_model=classifiers[0][1]).fit(train.iloc[:,1:], train.iloc[:,0])
test['stacking']=clf.predict(test.iloc[:,1:19])

In [37]:
# 評估各項模型好壞
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
result=pd.DataFrame(index=['accuracy','Survived=0 recall','Survived=1 recall','F1 score'],columns=test.columns[-8:])
for i, model in enumerate(test.columns[-8:]):
    p=precision_recall_fscore_support(test['Survived'], test[model])
    result.iloc[0,i]=accuracy_score(test['Survived'], test[model])
    result.iloc[1,i]=p[1][0]
    result.iloc[2,i]=p[1][1]
    result.iloc[3,i]=p[2][1]
result

,pred_Cart,pred_NBayes,pred_KNN,vote,pred_RandomForest,pred_RandomForest_2,pred_AdaBoost,stacking
accuracy,0.787709,0.776536,0.698324,0.793296,0.854749,0.854749,0.815642,0.782123
Survived=0 recall,0.849057,0.820755,0.830189,0.867925,0.933962,0.924528,0.877358,0.801887
Survived=1 recall,0.69863,0.712329,0.506849,0.684932,0.739726,0.753425,0.726027,0.753425
F1 score,0.728571,0.722222,0.578125,0.729927,0.80597,0.808824,0.76259,0.738255


In [ ]:
clf.base_models_[0][2].predict(test.iloc[:,1:19])

In [ ]:
a= [list() for x in clf.base_models]

In [ ]:
a

In [44]:
a=np.column_stack([
np.column_stack([model.predict(test.iloc[:,1:19]) for model in clf.base_models]).mean(axis=1 )
    for base_models in clf.base_models_ ])

In [49]:
clf.meta_model_.predict(a)==clf.meta_model_.predict(np.round(a))

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,